# covid-nsw
Driven by SQL, I explore [NSW COVID-19 cases by location](https://data.nsw.gov.au/search/dataset/ds-nsw-ckan-aefcde60-3b0c-4bc0-9af1-6fe652944ec2/details?q=).

## Setup
Here, I read in the input files into SQL-queriable tables.

In [1]:
import os
from glob import glob
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite://', echo=False)

covid_df = pd.read_csv(glob('./input/confirmed_cases*.csv')[0])
covid_df.to_sql('covid_nsw', 
                engine, 
                if_exists='replace', 
                index=False)

## Cleaning

### Inspecting

In [2]:
pd.read_sql_query("""SELECT *
                     FROM covid_nsw
                     LIMIT 10""", engine)

,notification_date,postcode,likely_source_of_infection,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
0,2020-01-25,2134,Overseas,X700,Sydney,11300,Burwood (A)
1,2020-01-25,2121,Overseas,X760,Northern Sydney,16260,Parramatta (C)
2,2020-01-25,2071,Overseas,X760,Northern Sydney,14500,Ku-ring-gai (A)
3,2020-01-27,2033,Overseas,X720,South Eastern Sydney,16550,Randwick (C)
4,2020-03-01,2077,Overseas,X760,Northern Sydney,14000,Hornsby (A)
5,2020-03-01,2163,Overseas,X710,South Western Sydney,12850,Fairfield (C)
6,2020-03-02,2073,Locally acquired - no links to known case or c...,X760,Northern Sydney,14500,Ku-ring-gai (A)
7,2020-03-02,2217,Overseas,X720,South Eastern Sydney,10500,Bayside (A)
8,2020-03-02,2077,Locally acquired - linked to known case or clu...,X760,Northern Sydney,14000,Hornsby (A)
9,2020-03-03,2196,Overseas,X700,Sydney,11570,Canterbury-Bankstown (A)


### Date Range
Exploring the first and last date of reporting period.

In [3]:
pd.read_sql_query("""SELECT MIN(notification_date) first_date,
                         MAX(notification_date) last_date
                     FROM covid_nsw""", engine)

,first_date,last_date
0,2020-01-25,2021-10-18


### Missing Values

#### Identifying & diagnosing
*NB: the `postcode` column has `None` instead of `NULL` values*

In [4]:
pd.read_sql_query("""SELECT *
                     FROM covid_nsw
                     WHERE notification_date IS NULL
                         OR postcode = 'None'
                         OR likely_source_of_infection IS NULL
                         OR lhd_2010_code IS NULL
                         OR lhd_2010_name IS NULL
                         OR lga_code19 IS NULL
                         OR lga_name19 IS NULL""", engine)

,notification_date,postcode,likely_source_of_infection,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
0,2020-03-07,2091,Locally acquired - linked to known case or clu...,None,None,None,None
1,2020-03-15,None,Overseas,None,None,None,None
2,2020-03-15,None,Interstate,None,None,None,None
3,2020-03-20,2612,Overseas,None,None,None,None
4,2020-03-20,None,Overseas,None,None,None,None
...,...,...,...,...,...,...,...
1130,2021-10-17,None,Locally acquired - investigation ongoing,X999,Correctional settings,X999,Correctional settings
1131,2021-10-17,None,Locally acquired - investigation ongoing,None,None,None,None
1132,2021-10-17,None,Locally acquired - investigation ongoing,None,None,None,None
1133,2021-10-18,None,Locally acquired - investigation ongoing,None,None,None,None


Finding the count of missing values in each column:

In [5]:
pd.read_sql_query("""SELECT SUM(CASE WHEN notification_date IS NULL THEN 1 ELSE 0 END) 'nd_miss',
                         SUM(CASE WHEN postcode IS 'None' THEN 1 ELSE 0 END) 'pc_miss', 
                         SUM(CASE WHEN likely_source_of_infection IS NULL THEN 1 ELSE 0 END) 'lsi_miss',
                         SUM(CASE WHEN lhd_2010_code IS  NULL THEN 1 ELSE 0 END) 'lhdc_miss',
                         SUM(CASE WHEN lhd_2010_name IS NULL THEN 1 ELSE 0 END) 'lhdn_miss',
                         SUM(CASE WHEN lga_code19 IS NULL THEN 1 ELSE 0 END) 'lgac_miss',
                         SUM(CASE WHEN lga_name19 IS NULL THEN 1 ELSE 0 END) 'lgan_miss'
                     FROM covid_nsw""", engine)

,nd_miss,pc_miss,lsi_miss,lhdc_miss,lhdn_miss,lgac_miss,lgan_miss
0,0,1071,0,964,964,964,964


Unfortunately, if the `postcode` is missing, it would be difficult to impute a value, as this was not published by Data.NSW. 

However, LGAs can be imputed from `postcode` in the following rows, where `postcode` was not `None`, but `lga_name19` was:

In [6]:
pd.read_sql_query("""SELECT *
                     FROM covid_nsw
                     WHERE postcode IS NOT 'None'
                         AND (lga_name19 IS NULL OR lga_code19 IS NULL)""", engine)

,notification_date,postcode,likely_source_of_infection,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
0,2020-03-07,2091,Locally acquired - linked to known case or clu...,None,None,None,None
1,2020-03-20,2612,Overseas,None,None,None,None
2,2020-04-30,2612,Locally acquired - linked to known case or clu...,None,None,None,None
3,2020-05-13,2602,Overseas,None,None,None,None
4,2020-06-07,2606,Overseas,None,None,None,None
...,...,...,...,...,...,...,...
59,2021-09-12,2607,Locally acquired - investigation ongoing,None,None,None,None
60,2021-09-16,2906,Locally acquired - investigation ongoing,None,None,None,None
61,2021-09-17,2902,Overseas,None,None,None,None
62,2021-10-02,2913,Interstate,None,None,None,None


Local Health Districts can also be imputed from `postcode` using NSW Health's [Wall Map](https://www.health.nsw.gov.au/lhd/Documents/lhd-wall-map.pdf). 

However, presently, we are not looking to analyse Local Health District information.

#### Resolving missing values
We will use [WorldPostalCodes'](https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-new-south-wales) table to convert NSW postcodes to the LGAs. 

In [7]:
postcode_to_lga_df = pd.read_csv('input/postcode_to_lga.csv') # importing csv

postcode_to_lga_df.to_sql('postcode_to_lga', engine, if_exists='replace')

postcode_to_lga_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postcode Code  616 non-null    int64 
 1   Postcode Name  616 non-null    object
 2   lga            616 non-null    object
dtypes: int64(1), object(2)
memory usage: 14.6+ KB


In [8]:
covid_nsw_df_2 = pd.read_sql_query("""SELECT c.*, 
                                       p.lga
                                   FROM covid_nsw c
                                   INNER JOIN postcode_to_lga p
                                       ON c.postcode = p.'Postcode code'""", engine)

covid_nsw_df_2.to_sql('covid_nsw_2', engine, if_exists='replace', index=False)

In [9]:
pd.read_sql_query("""SELECT *
                     FROM covid_nsw_2
                     ORDER BY notification_date
                     LIMIT 5""", engine)

,notification_date,postcode,likely_source_of_infection,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19,lga
0,2020-01-25,2134,Overseas,X700,Sydney,11300,Burwood (A),Burwood
1,2020-01-25,2121,Overseas,X760,Northern Sydney,16260,Parramatta (C),Parramatta
2,2020-01-25,2071,Overseas,X760,Northern Sydney,14500,Ku-ring-gai (A),Ku-ring-gai
3,2020-01-27,2033,Overseas,X720,South Eastern Sydney,16550,Randwick (C),Randwick
4,2020-03-01,2077,Overseas,X760,Northern Sydney,14000,Hornsby (A),Hornsby


The `lga` column in this table (`covid_nsw_2`) will be used instead of `lga_code19` to ensure LGA completeness.

## Exploratory Data Analysis
### Daily Count
#### Total daily reported cases
As the table has each row representing a notified COVID case, we will aggregate them by `notification_date`:

In [10]:
pd.read_sql_query("""SELECT notification_date,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 1""", engine)

,notification_date,cases
0,2020-01-25,3
1,2020-01-27,1
2,2020-03-01,2
3,2020-03-02,3
4,2020-03-03,6
...,...,...
537,2021-10-14,411
538,2021-10-15,330
539,2021-10-16,298
540,2021-10-17,281


#### Highest reported daily cases - top 10

In [11]:
pd.read_sql_query("""SELECT notification_date date,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 2 DESC
                     LIMIT 10""", engine)

,date,cases
0,2021-09-03,1524
1,2021-09-09,1515
2,2021-09-10,1452
3,2021-09-08,1424
4,2021-09-07,1397
5,2021-09-02,1358
6,2021-09-01,1325
7,2021-09-15,1305
8,2021-09-17,1280
9,2021-09-06,1268


#### "Zero" days
As the `covid_nsw_2` data only reports cases that have been notified, we will need to:
1. Generate the dates between the start and end of the reporting period. This will update automatically as we load in a new CSV from Data.NSW.
2. `LEFT JOIN` those dates with `covid_nsw_2` and find dates that were *not* in the `covid_nsw_2` table.

In [12]:
pd.read_sql_query("""WITH RECURSIVE dates(date) AS (
                         VALUES((
                             SELECT MIN(notification_date)
                             FROM covid_nsw_2
                             )
                         )
                         UNION ALL
                         SELECT DATE(date, '+1 day')
                         FROM dates
                         WHERE date < (
                             SELECT MAX(notification_date)
                             FROM covid_nsw_2
                             )
                     )
                     
                     SELECT d.date zero_days
                     FROM dates d
                     LEFT JOIN covid_nsw_2 c
                         ON d.date = c.notification_date
                     WHERE notification_date IS NULL
                     ORDER BY 1""", engine)

,zero_days
0,2020-01-26
1,2020-01-28
2,2020-01-29
3,2020-01-30
4,2020-01-31
...,...
86,2021-05-31
87,2021-06-01
88,2021-06-03
89,2021-06-11


### Geographic

#### "Greater Sydney"
The original dataset reports on *all* LGAs in NSW. In order to limit by "Greater Sydney", where needed, we will prepare a table that lists Greater Sydney LGAs.

Although there are some definition differences, we use the "Greater Sydney" LGAs as exhaustively defined in the COVID rules, published by the [NSW Government](https://www.nsw.gov.au/covid-19/stay-safe/protecting/advice-high-risk-groups/disability/local-councils-greater-sydney) and referenced in press conferences and media updates.

##### Importing, inspecting & cleaning
We will import & inspect a CSV that lists Greater Sydney LGAs, published by [NSW Government](https://www.nsw.gov.au/covid-19/stay-safe/protecting/advice-high-risk-groups/disability/local-councils-greater-sydney).

In [13]:
greater_sydney_lgas_df = pd.read_csv('input/greater_sydney_lgas.csv')

greater_sydney_lgas_df.to_sql('greater_sydney_lgas', engine, if_exists='replace')

greater_sydney_lgas_df

,lga_name
0,Bayside
1,Blacktown
2,Blue Mountains
3,Burwood
4,Camden
5,Campbelltown
6,Canada Bay
7,Canterbury-Bankstown
8,Central Coast
9,Cumberland


The LGA names in `greater_sydney_lgas` need to align with our main table, `covid_nsw_2`.

We will need to `LEFT JOIN` the two tables and identify LGAs in `greater_sydney_lgas` but not in `covid_nsw_2`. These will be the LGAs in Greater Sydney that will need to have their names amended to match those in `covid_nsw_2`.

In [14]:
pd.read_sql_query("""SELECT DISTINCT lga_name
                     FROM greater_sydney_lgas g
                     LEFT JOIN covid_nsw_2 c
                         ON g.lga_name = c.lga
                     WHERE lga IS NULL""", engine)

,lga_name
0,Hunter�s Hillv
1,The Hills Shire


Finding the way that `covid_nsw_2` refers to the identified Greater Sydney LGAs above:

In [15]:
pd.read_sql_query("""SELECT DISTINCT lga 
                     FROM covid_nsw_2
                     WHERE lga LIKE 'Hunter%'
                         OR lga LIKE '%Hills%'
                     ORDER BY 1 DESC""", engine)

,lga
0,Hunters Hill
1,Hills Shire


Updating the `greater_sydney_lgas` table to ensure it has the names in `covid_nsw_2`:

*NB: with `sqlalchemy`, both `UPDATE` and `CREATE TABLE` produce 'ResourceClosedError: This result object does not return rows. It has been closed automatically.' ([Source](https://github.com/sqlalchemy/sqlalchemy/issues/5433)). Thus, instead, we are using `UNION` so that later analysis can use `IN` for this table.*

In [16]:
greater_sydney_lgas_df_2 = pd.read_sql_query("""SELECT lga_name
                                             FROM greater_sydney_lgas
                                             UNION
                                             SELECT DISTINCT lga 
                                             FROM covid_nsw_2
                                             WHERE lga LIKE 'Hunter%'
                                                 OR lga LIKE '%Hills%'
                                             ORDER BY 1""", engine)

greater_sydney_lgas_df_2.to_sql('greater_sydney_lgas', engine, if_exists='replace', index=False)

pd.read_sql_query("""SELECT *
                     FROM greater_sydney_lgas""", engine)

,lga_name
0,Bayside
1,Blacktown
2,Blue Mountains
3,Burwood
4,Camden
5,Campbelltown
6,Canada Bay
7,Canterbury-Bankstown
8,Central Coast
9,Cumberland


#### LGAs
##### NSW - Top 10

In [17]:
pd.read_sql_query("""SELECT lga,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 2 DESC
                     LIMIT 10""", engine)

,lga,cases
0,Canterbury-Bankstown,12066
1,Cumberland,7872
2,Blacktown,7153
3,Liverpool,5174
4,Fairfield,4142
5,Penrith,3296
6,Parramatta,3262
7,Sydney,1960
8,Wollondilly,1924
9,Bayside,1923


#####  Greater Sydney - ranked

In [18]:
pd.read_sql_query("""SELECT lga,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     WHERE lga IN (
                         SELECT *
                         FROM greater_sydney_lgas
                     )
                     GROUP BY 1
                     ORDER BY 2 DESC""", engine)

,lga,cases
0,Canterbury-Bankstown,12066
1,Cumberland,7872
2,Blacktown,7153
3,Liverpool,5174
4,Fairfield,4142
5,Penrith,3296
6,Parramatta,3262
7,Sydney,1960
8,Wollondilly,1924
9,Bayside,1923


##### Total reported cases, during "Delta" - top 10 in NSW - compared with pre-"Delta" total cases
Although unclear of the exact date, the "Delta" wave (also known as Australia's 3rd wave, and NSW's 2nd wave), this likely began on or around 18 June 2021.

Source 1: [ABC News 2021a, 'Outbreak, How Australia lost control of the COVID Delta variant | ABC News'](https://www.youtube.com/watch?v=M9MPXCpyCnY&ab_channel=ABCNewsIn-depth)

Source 2: [ABC News 2021b, 'NSW Health records additional COVID-19 case, masks mandatory for public transport in Greater Sydney'](https://www.abc.net.au/news/2021-06-18/nsw-records-one-covid-19-case-masks-for-public-transport/100225370)

In [19]:
pd.read_sql_query("""WITH lga_top10_all AS (
                         SELECT lga,
                             COUNT(*) 'cases'
                         FROM covid_nsw_2
                         GROUP BY 1
                         ORDER BY 2 DESC
                         LIMIT 10
                     ),
                     
                     cases_pre_delta AS (
                         SELECT lga,
                             COUNT(*) 'cases_pre_delta'
                         FROM covid_nsw_2
                         WHERE notification_date < '2021-06-18'
                             AND lga IN
                                 (SELECT lga
                                  FROM lga_top10_all
                                 )
                         GROUP BY 1
                         ORDER BY 2 DESC
                     )
                     
                     SELECT l.lga,
                         c.cases_pre_delta,
                         (l.cases - c.cases_pre_delta) 'cases_post_delta',
                         l.cases 'cases_total'
                     FROM lga_top10_all l
                     LEFT JOIN cases_pre_delta c
                         ON l.lga = c.lga
                     ORDER BY 4 DESC""", engine)

,lga,cases_pre_delta,cases_post_delta,cases_total
0,Canterbury-Bankstown,275,11791,12066
1,Cumberland,236,7636,7872
2,Blacktown,238,6915,7153
3,Liverpool,139,5035,5174
4,Fairfield,134,4008,4142
5,Penrith,148,3148,3296
6,Parramatta,145,3117,3262
7,Sydney,228,1732,1960
8,Wollondilly,98,1826,1924
9,Bayside,116,1807,1923


##### No reported cases

In [20]:
pd.read_sql_query("""SELECT DISTINCT lga
                     FROM postcode_to_lga
                     WHERE lga NOT IN (
                         SELECT lga
                         FROM covid_nsw_2
                         )
                     ORDER BY 1""", engine)

,lga
0,Balranald
1,Bland
2,Coolamon
3,Gwydir
4,Lachlan
5,Leeton
6,Lockhart
7,Narrabri
